# DHT22 Prediction Model

아래 중간 중간 `#빈칸`을 채우며 적절히 설계해보세요. 끝까지 실행하면 `temp_predict.onnx` 및 `humi_predict.onnx` 파일이 생성됩니다.

In [ ]:
import json
import torch
import torch.nn as nn
import torch.onnx
import numpy as np
from torch.utils.data import DataLoader, Dataset

### Class Definitions

In [ ]:
class DHTDataset(Dataset):
    def __init__(self, key):
        self.data = []
        self.target = []
        self.key = key

        db = open("db.json", "r")
        humi_temp = json.load(db)

        for i in range(len(humi_temp.get(self.key))-62):
            tmp_data = []
            tmp_data.append([float(humi_temp.get(self.key)[i][1])])
            tmp_data.append([float(humi_temp.get(self.key)[i+1][1])])
            tmp_data.append([float(humi_temp.get(self.key)[i+2][1])])
            # 연속으로 3번 측정한 값
            self.data.append(torch.as_tensor(tmp_data))

            self.target.append( #1분뒤 값
                torch.as_tensor([float(humi_temp.get(self.key)[i+62][1])]))
            
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        data, target = self.data[index], self.target[index]
        return data, target

In [ ]:
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()

        self.linear = nn.Sequential(
            nn.Linear(3,1)
        )
        #3가지을 넣어 하나의 값을 받아오는

    def forward(self, x):
        x=self.linear(x.squeeze())
        return x

### Function Definitions

##### A function to split dataset into several data loaders

In [ ]:
def splitDataset(dataset):
    test_size = int(len(dataset)/6)
    val_size = int(len(dataset)/6)
    train_size = len(dataset) - test_size - val_size
#머신러닝셋 나누기
    trainset, valset, testset = torch.utils.data.random_split(
        dataset, [train_size, val_size, test_size])

    trainloader = DataLoader(trainset, batch_size=16, shuffle=True)
    validloader = DataLoader(valset, batch_size=16, shuffle=True)
    testloader = DataLoader(testset, batch_size=16, shuffle=True)

    return trainloader, validloader, testloader

##### A function to train model

In [ ]:
def train_model(model, patience, num_epochs, train_loader, valid_loader):
    train_losses = []
    valid_losses = []
    mean_train_losses = []
    mean_valid_losses = []
    p = 0
    min_valid_loss = float("inf")

    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(1, num_epochs + 1):

        model.train()
        for train_data, train_target in train_loader:
            if torch.cuda.is_available():
                device = torch.device("cuda")
                train_data = train_data.to(device, dtype=torch.float)
                train_target = train_target.to(device, dtype=torch.float)
            optimizer.zero_grad()
            output = model(train_data)
            loss = criterion(output, train_target)
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())

        model.eval()

        for valid_data, valid_target in valid_loader:
            if torch.cuda.is_available():
                device = torch.device("cuda")
                valid_data = valid_data.to(device, dtype=torch.float)
                valid_target = valid_target.to(device, dtype=torch.float)
            output = model(valid_data)
            loss = criterion(output, valid_target)
            valid_losses.append(loss.item())

        train_loss = np.mean(train_losses)
        valid_loss = np.mean(valid_losses)
        mean_train_losses.append(train_loss)
        mean_valid_losses.append(valid_loss)

        if min_valid_loss > valid_loss:
            min_valid_loss = valid_loss
            print(f"min_valid_loss: {min_valid_loss}")

        epoch_len = len(str(num_epochs))
        print_msg = (f"[{epoch:>{epoch_len}}/{num_epochs:>{epoch_len}}] " +
                     f"train_loss: {train_loss:.5f} " +
                     f"valid_loss: {valid_loss:.5f} ")
        print(print_msg)

        train_losses = []
        valid_losses = []

        if min_valid_loss < valid_loss and epoch > 1:
            p = p + 1
            print(f'patience: {p}')
        else:
            p = 0
            torch.save(model.state_dict(), "bestmodel.pt")
            print("Saving Model...")

        if patience == p:
            print("Early stopping")
            break

    model.load_state_dict(torch.load("bestmodel.pt"))

    return model

### Training

##### Load datasets and get dataloaders

In [ ]:
tempDataset = DHTDataset("temperature")
humiDataset = DHTDataset("humidity")

temp_trainloader, temp_validloader, temp_testloader = splitDataset(tempDataset)
humi_trainloader, humi_validloader, humi_testloader = splitDataset(humiDataset)

##### Make models

In [ ]:
tempModel = SimpleModel()
humiModel = SimpleModel()

if torch.cuda.is_available():
    tempModel = tempModel.cuda()
    humiModel = humiModel.cuda()

##### Do training with earlystopping

In [ ]:
temp_predict_model = train_model(tempModel, patience=30, num_epochs=1000,
                                 train_loader=temp_trainloader,
                                 valid_loader=temp_validloader)
torch.save(temp_predict_model.state_dict(), "temp_predict.pt")

min_valid_loss: 428.7687123616536
[   1/1000] train_loss: 505.73347 valid_loss: 428.76871 
Saving Model...
min_valid_loss: 308.77620188395184
[   2/1000] train_loss: 369.53416 valid_loss: 308.77620 
Saving Model...
min_valid_loss: 216.8365567525228
[   3/1000] train_loss: 263.24193 valid_loss: 216.83656 
Saving Model...
min_valid_loss: 147.85588200887045
[   4/1000] train_loss: 182.42872 valid_loss: 147.85588 
Saving Model...
min_valid_loss: 97.7727165222168
[   5/1000] train_loss: 122.78373 valid_loss: 97.77272 
Saving Model...
min_valid_loss: 62.68458271026611
[   6/1000] train_loss: 80.07968 valid_loss: 62.68458 
Saving Model...
min_valid_loss: 38.78587373097738
[   7/1000] train_loss: 50.54128 valid_loss: 38.78587 
Saving Model...
min_valid_loss: 23.167890866597492
[   8/1000] train_loss: 30.79594 valid_loss: 23.16789 
Saving Model...
min_valid_loss: 13.297178427378336
[   9/1000] train_loss: 18.09523 valid_loss: 13.29718 
Saving Model...
min_valid_loss: 7.371254920959473
[  10/100

In [ ]:
humi_predict_model = train_model(humiModel, patience=30, num_epochs=1000,
                                 train_loader=humi_trainloader,
                                 valid_loader=humi_validloader)
torch.save(humi_predict_model.state_dict(), "humi_predict.pt")

min_valid_loss: 169.66026306152344
[   1/1000] train_loss: 246.14263 valid_loss: 169.66026 
Saving Model...
min_valid_loss: 75.33166631062825
[   2/1000] train_loss: 118.55401 valid_loss: 75.33167 
Saving Model...
min_valid_loss: 29.043671131134033
[   3/1000] train_loss: 49.79583 valid_loss: 29.04367 
Saving Model...
min_valid_loss: 9.564021110534668
[   4/1000] train_loss: 18.04329 valid_loss: 9.56402 
Saving Model...
min_valid_loss: 2.719623565673828
[   5/1000] train_loss: 5.61129 valid_loss: 2.71962 
Saving Model...
min_valid_loss: 0.767499253153801
[   6/1000] train_loss: 1.56020 valid_loss: 0.76750 
Saving Model...
min_valid_loss: 0.2850678401688735
[   7/1000] train_loss: 0.46712 valid_loss: 0.28507 
Saving Model...
min_valid_loss: 0.20214230163643757
[   8/1000] train_loss: 0.23361 valid_loss: 0.20214 
Saving Model...
min_valid_loss: 0.19036231841892004
[   9/1000] train_loss: 0.17508 valid_loss: 0.19036 
Saving Model...
[  10/1000] train_loss: 0.17315 valid_loss: 0.19060 
pat

### ONNX Coversion

In [ ]:
def convert_to_onnx(saved_file_name):
    saved_model = SimpleModel()
    saved_model.load_state_dict(torch.load(saved_file_name))
    saved_model.eval()
    torch.onnx.export(
        saved_model,
        torch.randn((1, 3, 1)),
        saved_file_name[:-3] + '.onnx',
        opset_version=11,
        do_constant_folding=True,
        input_names=["input"],
        output_names=["output"]
    )

In [ ]:
convert_to_onnx("temp_predict.pt")
convert_to_onnx("humi_predict.pt")

In [ ]:
# End of Document.